In [ ]:
#Downloading and installing required packages
!pip install pytextrank
!python -m spacy download en_core_web_md
!pip install tika

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-04-26 14:32:02.497163: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 14:32:03.684534: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 18.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.

In [ ]:
import pytextrank
import spacy
import tika
#importc
import nltk.data
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from tika import parser
nltk.download('punkt')
from nltk import tokenize
string.punctuation
# Start running the tika service
tika.initVM()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from io import StringIO
from bs4 import BeautifulSoup
from tika import parser

In [ ]:
#this code for getting the online pdf content using tika tool
def gettext(link):
   file_data = []
   _buffer = StringIO()
   data = parser.from_file(link, xmlContent=True)
   xhtml_data = BeautifulSoup(data['content'])
   for page, content in enumerate(xhtml_data.find_all('div', attrs={'class': 'page'})):
       #print('Parsing page {} of pdf file...'.format(page+1))
      _buffer.write(str(content))
      parsed_content = parser.from_buffer(_buffer.getvalue())
      _buffer.truncate()
      file_data.append({'id': 'page_'+str(page+1), 'content': parsed_content['content']})
      print(file_data[page].get("content"))
   return(file_data[page].get("content"))


In [ ]:
#gettext("https://ayushmanup.in/admin/Clients/Doc/87_Guidelines-on-Processes-for-Hospital-Transaction.pdf")

#Loading model...

In [ ]:
#this function is for converting paragraph pdf content into sentences
def texttosen(data):

    text=tokenize.sent_tokenize(data)
    return(text)

def split(text):
  text = re.sub(r"\n{4,}","<stop>",var1)
  text = re.sub(r"\b[a-z]{1,3}\.","<stop>",text)
  text = re.sub(r"\b[a-zA-Z]\.\s{1,}(?!\d+/-)","<stop>",text)
  text = re.sub(r"\.\s{0,}(?=[A-Z])","<stop>",text)
  text = re.sub(r"\?","<stop>",text)
  text = re.sub(r"\!","<stop>",text)
  text = re.sub(r"\([^()]*\)", "",text)
  text = re.sub(r"(?<![a-z])(?<!/)[a-z]{1,}\)","<stop>",text)
  text = re.sub(r"\;","<stop>",text)
  text = re.sub(r"\:","<stop>",text)
  text = re.sub(r"\uf0d8","<stop>",text)
  sentence = re.split(r"<stop>",text)
  #print(sentence)
  return(sentence)

#Cleaning Data(scentences)....

In [ ]:
#defining the function to remove punctuations
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

In [ ]:
pattern = r"\[[^\]]*\]"
Rs = r"(?<!\d)\.(?!\d)"
Rs1 = r"(?<!Rs)\.(?!\s\d)"

#remove_noise function removes all the extra spaces or noise present in any sentence
def remov_noise(text):
  text = re.sub(r"\s+"," ", text)
  text = re.sub(r"\.","", text)
  text = re.sub(r"\bNote \b"+"[0-9]"+"\:","", text)
  text = re.sub(r"(.*\uf0a7|.*\uf0b7)","", text)
  text = re.sub(r"\([^()]*\)","",text)
  text = re.sub(pattern, "", text)
  text = re.sub(r"[a-z]\.","",text)
  text = re.sub(r"\$","",text)
  text = re.sub(r"\-","",text)
  text = re.sub(r"[a-z]*\)","",text)
  text = re.sub(r"\’","",text)
  text = re.sub(r"\s+"," ",text)
  return(text)

#lowercase function for converting the uppercase sentences into lowercase
def lowercase(sentence):
   sentence = re.sub(r"[A-Z]", lambda match: match.group(0).lower(), sentence)
   return(sentence)

#This code get proper sentences

In [ ]:
nlp = spacy.load("en_core_web_md")

In [ ]:
#proper_sentences function get the sentences which have proper subject verb and object.This avoids useless sentences(which does not have proper subjects)
def proper_sentences(list1):
  list3 = []
  flag1 =0
  flag2 =0
  flag3 =0
  for item in list1:
    doc = nlp(item) #changed from item[0]
    for token in doc:
        if token.dep_ == 'nsubj' or token.dep_ == 'nsubjpass' or token.dep_ == 'nummod':
            flag1 = 1
        if token.dep_ == 'ROOT':
            flag2 = 1
        if token.dep_ == 'pobj' or token.dep_ == 'dobj':
            flag3 = 1
    if flag1 == 1:
      if flag2 == 1:
        if flag3 == 1:
            list3.append(item)
            flag1=flag2=flag3=0
    flag1=flag2=flag3=0
  return(list3)

#Entity replacement

In [ ]:
def enetity_resolution(text):
  text = re.sub(r"\b empanelled healthCare provider \b", " ehcp ", text)
  text = re.sub(r"\b state health agency \b", " sha ", text)
  text = re.sub(r"\b ams \b"," ayushman mitras ",text)
  text = re.sub(r"\b aadhaar card/number \b", " aadhar card ", text)
  text = re.sub(r"\b pmrssm e-card/id \b", " pmrssm e-card ", text)
  text = re.sub(r"\b operator \b", " ayushman mitra ", text)
  text = re.sub(r"\b insurance company/ trust \b", " insurance company ", text)
  text = re.sub(r"\b pmrssm system/state it system \b", " pmrssm system ", text)
  return(text)

In [ ]:
regsen = '''SHA and insurance company/ trust will need to have very close monitoring of the process in
order to ascertain challenges, if any, being faced and resolution of the same.'''
regEX = enetity_resolution(regsen)
print(regEX)

SHA and insurance company will need to have very close monitoring of the process in
order to ascertain challenges, if any, being faced and resolution of the same.


#Extracting Entities

In [ ]:
nlp = spacy.load("en_core_web_md")

In [ ]:
#text = "The scheme is an entitlement based and entitlement of the beneficiary is decided on the basis of family being figured in SECC database"
def entityhead(text):
  nounlist = ["NOUN","PROPN","ADP","ADJ","DET","PRON","PART"]
  deplist = ["nsubj","nsubjpass"]
  word = ""
  dependency = ""
  entites=[]
  entites1=[]
  entites2=[]
  flag = bool
  doc = nlp(text)
  #print(len(doc))
  for i in range(len(doc)):
    if doc[i].pos_ in nounlist:
       if doc[i].dep_ in deplist:
          dependency = doc[i].dep_
       word = word+" "+doc[i].text
    else :
       entites.append([word,dependency])
       word = ""
       dependency = ""
  entites.append([word,dependency])
  #print(entites)
  return(entites)


In [ ]:
def relations(text):
  verblist = []
  verblist1 = []
  pre_token = ""
  pre_pos_ = ""
  word = ""
  word1 = ""
  word2 = ""
  word3 = ""
  word4 = ""
  subobj = ["NOUN","PROPN","DET","NUM","CCONJ"]
  ##for checking the verb in  sentence...
  def verbcheck(i):
        word = i.split()
        for l in range(len(word)):
            if word[l] == "VERB":
                return(True)

  ## for getting the relations...........
  doc = nlp(text)
  for token in doc:
    if token.pos_ not in subobj :
      if pre_pos_ == "NOUN" or pre_pos_ == "PROPN":
        word = pre_token+" "+pre_pos_+" "+word+" "+token.text+" "+token.pos_
      elif pre_pos_!= "NOUN":
        word = word+" "+token.text+" "+token.pos_
    if token.pos_ in subobj:
      verblist.append(word)
      word = " "
    pre_token = token.text
    pre_pos_ = token.pos_

  ## verb filtering......................
  word2 = text.split()
  collect = []
  for i in verblist:
    flag = verbcheck(i)
    if flag == True:
        word3 = i.split()
        for j in range(len(word3)):
            if word3[j] == "NOUN" or word3[j] == "PROPN":
                collect.append(word3[j])
                continue
            if word3[j] in word2:
                collect.append(word3[j])
        for k in range(len(collect)):
          word4 = word4+" "+collect[k]
        verblist1.append(word4)
        collect = []
        word4 = ""
  return(verblist1)

In [ ]:
def nouncheck1(entites):
    entites1 = []
    flag = bool
    for i in range(len(entites)):
      doc =nlp(entites[i][0])
      for token in doc:
          if token.pos_ == "NOUN" or token.pos_ == "PROPN":
              flag = True
      if flag ==True:
          flag = bool
          if entites[i][0] != "":
            entites1.append(entites[i])
    return(entites1)

def lastprepremove(entites):
   entites1 = []
   word1 = ""
   for i in entites:
     word = i[0].split()
     doc = nlp(word[-1])
     if doc[0].pos_ == "ADP":
       word.pop(-1)
       #print(word)
       for j in range(len(word)):
          word1 = word1+" "+word[j]
       entites1.append([word1,i[1]])
       word1 = ""
       continue
     entites1.append(i)
     word1 =""
     #print(entites1)
   return(entites1)

In [ ]:
import re
def noise(text):
   text = re.sub(r"\s+", " ",text)
   text = re.sub(r"^\s", "",text)
   pattern = r"\s+(’s)" # This pattern looks for one or more spaces before the apostrophe and 's'
   replacement = r"\1" # This replaces the space and apostrophe 's' with just 's'
   text = re.sub(pattern, replacement, text)
   return(text)

In [ ]:
def clean(list):
  list1=[]
  for i in list:
   #print(i)
   i[0] = noise(i[0])
   list1.append(i)
  return(list1)

def clean1(list):
  list1=[]
  for i in list:
   #print(i)
   i = noise(i)
   list1.append(i)
  return(list1)

In [ ]:
def position(word,text):
   start_index = text.find(word)
   #print(end_index)
   return(start_index)

def getposition(entites,text):
   list1 = []
   #position1 = int
   for i in range(len(entites)):
     position1 = position(entites[i][0],text)
     #print(position1)
     list1.append([entites[i][0],entites[i][1],position1])
   return(list1)

In [ ]:
def prepcheck(text):
    doc = nlp(text)
    flag1 = bool;
    flag2 = bool;
    flag3 = bool;
    check = []
    pattern = ["NOUN","VERB","ADP"]
    pattern1 = ["PROPN","VERB","ADP"]
    pattern2 = ["NOUN","ADP","VERB"]
    pattern3 = ["PROPN","ADP","VERB"]
    for i in range(len(doc)):
        if i < 3:
          check.append(doc[i].pos_)
    if check == pattern or check == pattern1:
        #print("im in True1:",text)
        return(True)
    elif check == pattern2 or check == pattern3:
        #print("im in True2:",text)
        return(True)
    elif check !=pattern and check != pattern1:
        #print("im in False:",text)
        return(False)


def triplets(verb,entites,text,dummy):
    verb1=""
    subj = dummy

    def nouncheck(text):
       doc = nlp(text)
       if doc[0].pos_ == "NOUN" or doc[0].pos_ == "PROPN":
          return(True)
       return(False)

    if prepcheck(verb) == False:
       #print(prepcheck(verb),verb)
       #print(len(verb.split()))
       verbposition = position(verb,text)
       if len(verb.split()) == 1:
          #print(verb)
          verbposition = position(verb,text)
          for i in range(len(entites)):
              entitypositon = position(entites[i][0],text)
              if entites[i][1] =="nsubj" or entites[i][1] =="nsubjpass":
                 #print(verb)
                 if entites[i][2] < verbposition:
                    subj = entites[i][0]
              if entitypositon > verbposition:
                 #print(verb)
                 obj = entites[i][0]
                 #print(subj,verb,obj)
                 return(subj,verb,obj)

       elif nouncheck(verb) == True: # for skipping the nouns
          word = verb.split()
          for m in range(1,len(word)):
              verb1=verb1+" "+word[m]
          #print(verb1)
          verbposition = position(verb1,text)
          for i in range(len(entites)):
              entitypositon = position(entites[i][0],text)
              if entites[i][1] =="nsubj" or entites[i][1] =="nsubjpass":
                 if entites[i][2] < verbposition:
                    subj = entites[i][0]
              if entitypositon > verbposition:
                  obj = entites[i][0]
                  return(subj,verb1,obj)

       elif nouncheck(verb) == False:
          for i in range(len(entites)):
            entitypositon = position(entites[i][0],text)
            if entites[i][1] =="nsubj" or entites[i][1] =="nsubjpass":
              if entites[i][2] < verbposition:
                  subj = entites[i][0]
            if entitypositon > verbposition:
                  obj = entites[i][0]
                  return(subj,verb,obj)


    if prepcheck(verb) == True: ###this works absolutlyy for immediate prep after verb....
       word = verb.split()
       #print('sai')
       #print(word)
       for l in range(len(entites)):
           word1 = entites[l][0].split()
           #print(word1)
           if word1[-1] == word[0]:
              #print("sai")
              subj =entites[l][0]
              obj = entites[l+1][0]
              word.pop(0)
              #word.pop(1)
              for m in range(len(word)):
                  verb1=verb1+" "+word[m]
              #print('sai')
              return(subj,verb1,obj)

In [ ]:
def remove_leading_ADP(finallist):
  finallist2=[]
  for i in finallist:
    word3=""
    word1=""
    doc = nlp(i[1])
    if len(doc) == 1:
       finallist2.append(i)
    else:
      for token in doc:
        if token.pos_=="NOUN" or token.pos_=="PROPN":
            continue
        if token.pos_=="ADJ":
            continue
        word1=word1+" "+token.text
      word1.split()
      word2=i[2].split()
      doc = nlp(i[2]) #newly added
      if doc[-1].pos_ == "PRON":#
         word2.pop(-1)#
      for j in range(len(word2)):
          if word2[j] in word1:
              continue
          word3= word3+" "+word2[j]
      finallist2.append([i[0],word1,word3])
      word3= ""
  return(finallist2)

In [ ]:
def triples(text):
    subj = ""
    verb = ""
    obj =  ""
    dummy = ""
    fentityset = []

    #GETTING THE ENTITES......
    entites = entityhead(text)
    entites1 = nouncheck1(entites)
    entites1 = clean(entites1)
    entites1 = lastprepremove(entites1)
    entites1 = getposition(entites1,text)

    #GETTING THE RELATIONS....
    verblist = relations(text)
    verblist1 = clean1(verblist)

    #GETTING THE MAIN SUBJECT IN THE SENTENCE.....
    for i in entites1:
      if i[1] == "nsubj" or i[1] == "nsubjpass":
          dummy = i[0]
          break

    finallist = []
    finallist2 = []
    verb1 =""
    #MAKING THE TRIPLES OUT OF SENTENCE............
    for i in range(len(verblist1)):
      verb = verblist1[i]
      word3 = verb.split()
      verb1 =""
      if verblist1[i]!='':
        for j in range(len(word3)):
            if word3[j] == "NOUN" or word3[j] == "PROPN":
              continue
            verb1 = verb1+" "+word3[j]
        verb1 = noise(verb1)
        orlist = triplets(verb1,entites1,text,dummy)
        #print(orlist)
        if orlist !=None:
          finallist.append([orlist[0],orlist[1],orlist[2]])
        # again filter the entites repeated from relation sentence
        finallist2 = remove_leading_ADP(finallist)
    return(finallist2)

#Model for extracting sentences

In [ ]:
var1 = gettext("https://ayushmanup.in/admin/Clients/Doc/84_Guidelines-on-Claim-Settlement.pdf")
dataframe = texttosen(var1)#text)#converting the text into scentences and storing scentences in dataframe#https://ayushmanup.in/admin/Clients/Doc/84_Guidelines-on-Claim-Settlement.pdf

'''#Converting the extracted pdf content itno sentences........................................................
sentence1 = split(var1)
sentence2 = []
for i in sentence1:
  if i!= '':
    sentence2.append(i)
#print(sentence2)
sentence2 = proper_sentences(sentence2)
#print(sentence2)
sen = {
        'Sentences' : sentence2
}

##..............converting the dictionary into dataframe.......................................................
df5=df4=df3=df2=df1=df = pd.DataFrame(sen)
df2['Sentences']= df1['Sentences'].apply(lambda x:remov_noise(x))
#df3['Sentences']= df2['Sentences'].apply(lambda x:lowercase(x))
#df4['Sentences']= df3['Sentences'].apply(lambda x:remove_punctuation(x))'''

2023-04-26 14:32:35,915 [MainThread  ] [INFO ]  Retrieving https://ayushmanup.in/admin/Clients/Doc/84_Guidelines-on-Claim-Settlement.pdf to /tmp/admin-clients-doc-84_guidelines-on-claim-settlement.pdf.
INFO:tika.tika:Retrieving https://ayushmanup.in/admin/Clients/Doc/84_Guidelines-on-Claim-Settlement.pdf to /tmp/admin-clients-doc-84_guidelines-on-claim-settlement.pdf.
2023-04-26 14:32:36,225 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
2023-04-26 14:32:36,671 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
INFO:tika.tika:Retrieving http://search












Guidelines for Pradhan Mantri Rashtriya Swasthya Suraksha Mission (PMRSSM) 






































PMRSSM GUIDELINES  


ON  


CLAIM SETTLEMENT  
 
























































 
















Guidelines for Pradhan Mantri Rashtriya Swasthya Suraksha Mission (PMRSSM) 






































PMRSSM GUIDELINES  


ON  


CLAIM SETTLEMENT  
 
























































 






Guidelines for Pradhan Mantri Rashtriya Swasthya Suraksha Mission (PMRSSM) 


1. Guidelines on Claim Settlement  





All Empanelled Health Care Providers (EHCP) will make use of IT system of PMRSSM to 


manage the claims related transactions. IT system of PMRSSM has been developed for 


online transactions and all stakeholders are advised to maintain online transactions 


preferably to ensure the claim reporting in real time. However, keeping in mind the 


connectivity constraints faced by some districts an offl

"#Converting the extracted pdf content itno sentences........................................................ \nsentence1 = split(var1) \nsentence2 = []\nfor i in sentence1:\n  if i!= '':\n    sentence2.append(i)\n#print(sentence2)\nsentence2 = proper_sentences(sentence2)\n#print(sentence2)\nsen = {\n        'Sentences' : sentence2    \n} \n\n##..............converting the dictionary into dataframe.......................................................\ndf5=df4=df3=df2=df1=df = pd.DataFrame(sen)\ndf2['Sentences']= df1['Sentences'].apply(lambda x:remov_noise(x))\n#df3['Sentences']= df2['Sentences'].apply(lambda x:lowercase(x))\n#df4['Sentences']= df3['Sentences'].apply(lambda x:remove_punctuation(x))"

In [ ]:
#Converting the extracted pdf content itno sentences........................................................
sentence1 = split(var1)
print(sentence1)
sentence2 = []
for i in sentence1:
  if i!= '':
    sentence2.append(i)
#print(sentence2)
sentence2 = proper_sentences(sentence2)
#print(sentence2)
sen = {
        'Sentences' : sentence2
}

##..............converting the dictionary into dataframe.......................................................
df5=df4=df3=df2=df1=df = pd.DataFrame(sen)
df2['Sentences']= df1['Sentences'].apply(lambda x:remov_noise(x))
#df3['Sentences']= df2['Sentences'].apply(lambda x:lowercase(x))
df3['Sentences']= df2['Sentences'].apply(lambda x:remove_punctuation(x))

['', 'Guidelines for Pradhan Mantri Rashtriya Swasthya Suraksha Mission  ', 'PMRSSM GUIDELINES  \n\n\nON  \n\n\nCLAIM SETTLEMENT  \n ', ' ', 'Guidelines for Pradhan Mantri Rashtriya Swasthya Suraksha Mission  \n\n\n1', 'Guidelines on Claim Settlement  ', 'All Empanelled Health Care Providers  will make use of IT system of PMRSSM to \n\n\nmanage the claims related transactions', 'IT system of PMRSSM has been developed for \n\n\nonline transactions and all stakeholders are advised to maintain online transactions \n\n\npreferably to ensure the claim reporting in real time', 'However, keeping in mind the \n\n\nconnectivity constraints faced by some districts an offline arrangement has also been \n\n\nincluded in the IT system that has to be used only when absolute', 'The PMRSSM strives to \n\n\nmake the entire claim management paperless that is at any stage of claim registration, \n\n\nintimation, payment, investigation by EHCP or by the Trust/Insurer the need of submission \n\n\nof a phys

In [ ]:
print(df3.values[:300])

[['Guidelines for Pradhan Mantri Rashtriya Swasthya Suraksha Mission 1']
 ['All Empanelled Health Care Providers will make use of IT system of PMRSSM to manage the claims related transactions']
 ['IT system of PMRSSM has been developed for online transactions and all stakeholders are advised to maintain online transactions preferably to ensure the claim reporting in real time']
 ['However keeping in mind the connectivity constraints faced by some districts an offline arrangement has also been included in the IT system that has to be used only when absolute']
 ['The PMRSSM strives to make the entire claim management paperless that is at any stage of claim registration intimation payment investigation by EHCP or by the TrustInsurer the need of submission of a physical paper shall not be required']
 ['This mean that this claim data will be sent electronically through IT system to the Central State server']
 ['The NHA SHA Insurer  and EHCP shall be able to access this data with respect to 

#Model for generating triples

In [ ]:
finalist3= []
for i in df5.index:
  finalist3.append([df5["Sentences"][i],("\n\n"),triples(df5["Sentences"][i])])

In [ ]:
for i in finalist3:
  print(i)
print(len(finalist3))

['Guidelines for Pradhan Mantri Rashtriya Swasthya Suraksha Mission 1', '\n\n', []]
['All Empanelled Health Care Providers will make use of IT system of PMRSSM to manage the claims related transactions', '\n\n', [['All Empanelled Health Care Providers', '   will make', ' use of IT system of PMRSSM'], ['All Empanelled Health Care Providers', '   to manage', ' the claims related transactions']]]
['IT system of PMRSSM has been developed for online transactions and all stakeholders are advised to maintain online transactions preferably to ensure the claim reporting in real time', '\n\n', [['IT system of PMRSSM', '   has been developed for', ' online transactions'], ['all stakeholders', '   are advised to maintain online', ' the claim'], ['all stakeholders', '   preferably to ensure', ' the claim'], ['all stakeholders', ' claim reporting in', ' real time']]]
['However keeping in mind the connectivity constraints faced by some districts an offline arrangement has also been included in the IT

In [ ]:
print(finalist3[10])

['Any rejection notice issued by the TrustInsurer or the agency to EHCP shall clearly state that rejection is subject to the EHCPs right to appeal against rejection of the claim ', '\n\n', [['Any rejection notice', '   issued by', ' the TrustInsurer'], ['rejection', '   shall clearly state that', ' subject to the EHCPs right'], ['rejection', '   to appeal against', ' rejection of the claim']]]


In [ ]:
from tqdm import tqdm

finalist3 = []

for i in tqdm(df5.index, desc="Triplet extraction"):
    finalist3.append([df5["Sentences"][i], ("\n\n"), triples(df5["Sentences"][i])])


Triplet extraction: 100%|██████████| 42/42 [00:07<00:00,  5.49it/s]
